In [ ]:
# !pip install kaggle
# from google.colab import files
# files.upload()

In [ ]:
# !ls -1ha kaggle.json

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# # Permission Warning 이 일어나지 않도록 
# !chmod 600 ~/.kaggle/kaggle.json
# # 본인이 참가한 모든 대회 보기 
# !kaggle competitions list

In [ ]:
# !pip3 install tez

In [ ]:
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip3 install tokenizers wandb sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !pip3 install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

In [ ]:
import os
os.chdir("drive/")
os.chdir('My Drive')
os.chdir('Kaggle')
os.chdir('Feedback')

In [ ]:
# !kaggle competitions download -c feedback-prize-effectiveness
# !unzip feedback-prize-effectiveness.zip
# !ls
# !unzip fb_train.zip
# !unzip fb_test.zip

In [ ]:
OUTPUT_DIR = './feedback-deberta-v3-outputs/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jul 19 06:48:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# ====================================================
# CFG
# ====================================================

class GpuOptimizeConfig:
    fp16=True
    freezing=True
    optim8bit=True
    gradient_checkpoint=True

class CFG:
    wandb=True
    competition='Feedback'
    _wandb_kernel='bluehills'
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    model="microsoft/deberta-v3-large"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=50
    epochs=5
    encoder_lr=2e-5 #2e-5
    decoder_lr=2e-5 #2e-5
    min_lr=1e-6 
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=4
    fc_dropout=0.1
    target_size=1 
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    use_predefined_df=True
    merge_prev_df=False
    gpu_optimize_config=GpuOptimizeConfig
    val_steps=375
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [ ]:
# CFG.gpu_optimize_config['fp16'] = True
# CFG.gpu_optimize_config['freezing'] = True
# CFG.gpu_optimize_config['optim8bit'] = True
# CFG.gpu_optimize_config['gradient_checkpoint'] = True

In [ ]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    import wandb
    try:
        # from kaggle_secrets import UserSecretsClient
        # user_secrets = UserSecretsClient()
        # secret_value_0 = user_secrets.get_secret("wandb_api")
        # wandb.login(key=secret_value_0)
        print('login to wandb')
        wandb.login()
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

 
    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='Feedback-Training', 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="train",
                     anonymous=anony)

login to wandb


wandb: Currently logged in as: bluehills. Use `wandb login --relogin` to force relogin


# Library

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
from joblib import Parallel, delayed
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm

# Sklearn Imports
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder

# PyTorch
import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils.checkpoint import checkpoint
import torch.cuda.amp as amp

# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp
# import torch_xla.debug.metrics as met


# For Transformer Models
import transformers
from transformers import DataCollatorWithPadding

import tokenizers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig, AdamW, AutoModelForSequenceClassification, AutoModelForTokenClassification
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.__version__: 1.12.0+cu113
tokenizers.__version__: 0.12.1
transformers.__version__: 4.20.1
env: TOKENIZERS_PARALLELISM=true


In [ ]:
from datasets import Dataset, load_metric

In [ ]:
from tez import Tez, TezConfig
from tez.callbacks import EarlyStopping

# Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# Data Loading

In [ ]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
print(f"train.shape: {train.shape}")
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(train.head())
display(test.head())
display(submission.head())

train.shape: (36765, 5)
test.shape: (10, 4)
submission.shape: (10, 4)


,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


,discourse_id,essay_id,discourse_text,discourse_type
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. ...,Lead
1,5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person ma...,Position
2,9790d835736b,D72CB1C11673,it can decrease stress levels,Claim
3,75ce6d68b67b,D72CB1C11673,a great chance to learn something new,Claim
4,93578d946723,D72CB1C11673,can be very helpful and beneficial.,Claim


,discourse_id,Ineffective,Adequate,Effective
0,a261b6e14276,0.20,0.60,0.40
1,5a88900e7dc1,3.00,6.00,1.00
2,9790d835736b,1.00,2.00,3.00
3,75ce6d68b67b,0.33,0.34,0.33
4,93578d946723,0.01,0.24,0.47


In [ ]:
labels = ['Adequate', 'Effective', 'Ineffective']
leads = train.query('discourse_type == "Lead"')
leads.query("discourse_effectiveness == 'Effective'").iloc[0].discourse_text

'Limiting the usage of cars has personal and professional support all across the globe and yet it has yet to be embraced everywhere. Statistical proof show where it may help and real life examples of some of the effects of reducing, or getting rid of altogether, cars in one\'s daily life. While "recent studies suggest that Americans are buying fewer cars, driving less and getting fewer licenses as each year goes by" (Source 4), is that really enough or for the right reason? There are plenty of reasons to stop, or limit, the amount of cars being driven on the roads for every kind of person, from the hippie to the businessman, from the mom to the college student. '

In [ ]:
discourse_types = train.discourse_type.unique()
train['text_size'] = train.discourse_text.apply(lambda x: len(x))
train['word_count'] = train.discourse_text.apply(lambda x: len(x.split()))

In [ ]:
train_2021_preds_df = pd.read_csv('./train_2021_preds.csv')
train_2021_preds_df = train_2021_preds_df[train_2021_preds_df.in_2022 == False]

train_2021_preds_df.head()

,essay_id,discourse_id,discourse_text,discourse_type,in_2022,Adequate,Effective,Ineffective,discourse_effectiveness
10,A8445CABFECE,1.622576e+12,Drivers should not be able to use phones while...,Position,False,0.843945,0.132275,0.023645,Adequate
11,A8445CABFECE,1.622576e+12,Drivers who used their phone while operating a...,Claim,False,0.306641,0.685449,0.007604,Effective
12,A8445CABFECE,1.622576e+12,According to an article by the Edgar Snyder Fi...,Evidence,False,0.123401,0.870898,0.005660,Effective
13,A8445CABFECE,1.622576e+12,"In conclusion, drivers should not able to work...",Concluding Statement,False,0.869727,0.016228,0.114124,Adequate
14,6B4F7A0165B9,1.622644e+12,The ability to stay connected to people we kno...,Lead,False,0.091742,0.903809,0.004618,Effective


In [ ]:
train_2021_preds_df['label_prob'] = train_2021_preds_df[labels].max(axis=1)
len(train_2021_preds_df[['essay_id', 'label_prob']].groupby('essay_id').mean().sort_values('label_prob', ascending=False))

11403

In [ ]:
num_essays = 1000 # total 11403

confident_essays = train_2021_preds_df[['essay_id', 'label_prob']].groupby('essay_id').mean().sort_values('label_prob', ascending=False)[:num_essays]
essay_ids = set(confident_essays.index)
train_2021_filt_df = train_2021_preds_df[train_2021_preds_df.essay_id.isin(essay_ids)].reset_index(drop=True)

In [ ]:
train_2021_filt_df.head()

,essay_id,discourse_id,discourse_text,discourse_type,in_2022,Adequate,Effective,Ineffective,discourse_effectiveness,label_prob
0,A8445CABFECE,1.622576e+12,Drivers should not be able to use phones while...,Position,False,0.843945,0.132275,0.023645,Adequate,0.843945
1,A8445CABFECE,1.622576e+12,Drivers who used their phone while operating a...,Claim,False,0.306641,0.685449,0.007604,Effective,0.685449
2,A8445CABFECE,1.622576e+12,According to an article by the Edgar Snyder Fi...,Evidence,False,0.123401,0.870898,0.005660,Effective,0.870898
3,A8445CABFECE,1.622576e+12,"In conclusion, drivers should not able to work...",Concluding Statement,False,0.869727,0.016228,0.114124,Adequate,0.869727
4,AC594194F01C,1.622387e+12,I am typing about why drivers should not use p...,Lead,False,0.885059,0.057031,0.058350,Adequate,0.885059


In [ ]:
train.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,text_size,word_count
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,317,67
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,210,41
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,105,21
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,362,72
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,101,18


In [ ]:
import gc

TRAIN_DIR = "./train"
TEST_DIR = "./test"

TRAIN_DIR_2 = './fb_train'
TEST_DIR_2 = './fb_test'

def get_essay(essay_id, target_dir=TRAIN_DIR):
    essay_path = os.path.join(target_dir, f"{essay_id}.txt")
    essay_text = open(essay_path, 'r').read()
    return essay_text

def get_essay_2021(essay_id):
    return get_essay(essay_id, target_dir=TRAIN_DIR_2)


if CFG.use_predefined_df:
    del train_2021_filt_df
    del train_2021_preds_df
    gc.collect()
    train_df = pd.read_csv('all_train_df.csv')
else:
    train['essay_text'] = train['essay_id'].apply(get_essay)

    cols = ['discourse_id',	'essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'essay_text']
    train_df = train[cols]
    
    if CFG.merge_prev_df:
        train_2021_filt_df['essay_text'] = train_2021_filt_df['essay_id'].apply(get_essay_2021)
        train_2021_filt_df = train_2021_filt_df[cols]
        train_df = pd.concat([train_df, train_2021_filt_df], ignore_index=True)

    print(train_df.shape)
    display(train_df.head())
    train_df.to_csv('./train_df.csv', index=False)

In [ ]:
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,essay_text
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."


# CV

In [ ]:
cv = GroupKFold(n_splits=CFG.n_fold)

train_df['fold'] = -1
for fold_num, (train_idxs, test_idxs) in enumerate(cv.split(train_df.index, train_df.discourse_effectiveness, train_df.essay_id)):
    train_df.loc[test_idxs, ['fold']] = fold_num

# Tokenizer

In [ ]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model, use_fast=True)
tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Dataset

In [ ]:
LABEL_MAPPING = {"Ineffective": 0, "Adequate": 1, "Effective": 2}
# LABEL_MAPPING = {"Ineffective": [1., 0., 0.], "Adequate": [0., 1., 0.], "Effective": [0., 0., 1.]}
train_df['discourse_effectiveness']  = train_df['discourse_effectiveness'].map(lambda x : LABEL_MAPPING[x])
train_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,essay_text,fold
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,1,"Hi, i'm Isaac, i'm going to be writing about h...",4
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,1,"Hi, i'm Isaac, i'm going to be writing about h...",4
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,1,"Hi, i'm Isaac, i'm going to be writing about h...",4
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,1,"Hi, i'm Isaac, i'm going to be writing about h...",4
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,1,"Hi, i'm Isaac, i'm going to be writing about h...",4


In [ ]:
# ====================================================
# Define max_len
# ====================================================
# lengths_dict = {}

# for text_col in ['discourse_text', 'essay_text', 'discourse_type']:
#     lengths = []
#     tk0 = tqdm(train_df[text_col].fillna("").values, total=len(train_df))
#     for text in tk0:
#         length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
#         lengths.append(length)
#     lengths_dict[text_col] = lengths
    
# discourse_len = max(lengths_dict['discourse_text']) + max(lengths_dict['discourse_type'])
# essay_len = max(lengths_dict['essay_text'])
# CFG.max_len = max(discourse_len, essay_len) + 4 # CLS + SEP + SEP + SEP
CFG.max_len = 512
CFG.essay_max_len = 512
LOGGER.info(f"max_len: {CFG.max_len}")

max_len: 512


In [ ]:
# train_df['text'] = train_df['discourse_type'] + '[SEP]' + train_df['discourse_text'] + '[SEP]' + train_df['essay_text']
train_df['text'] = train_df['discourse_type'] + '[SEP]' + train_df['discourse_text']
train_df['essay_text'] = '[SEP]' + train_df['essay_text']
# train_df['text'] = train_df['discourse_type'] + ' ' + train_df['discourse_text']

In [ ]:
LABEL_MAPPING = {"Ineffective": 0, "Adequate": 1, "Effective": 2}

In [ ]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text, essay_text=None):
    text = text[:cfg.max_len]
    if essay_text:
        essay_text = essay_text[:cfg.essay_max_len]
        inputs = cfg.tokenizer.encode_plus(
            text,
            essay_text,
            add_special_tokens=True,
            max_length=cfg.max_len,
            padding="max_length",
            return_attention_mask=True,
            return_offsets_mapping=False
        )
    else:
        inputs = cfg.tokenizer(text,
                              add_special_tokens=True,
                              max_length=cfg.max_len,
                              padding="max_length",
                              return_attention_mask=True,
                              return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


def prepare_essay_input(cfg, text):
    text = text[:cfg.essay_max_len]
    inputs = cfg.tokenizer(text,
            add_special_tokens=True,
            max_length=cfg.essay_max_len,
            padding="max_length",
            return_attention_mask=True,
            return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs



class FeedbacksDataset(Dataset):
    def __init__(self, cfg, samples):
        self.cfg = cfg
        self.samples = samples
        self.texts = samples['text'].values
        self.essay_text = samples['essay_text'].values
        self.labels = samples['discourse_effectiveness'].values
        self.tokenizer = cfg.tokenizer

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        input_ids = prepare_input(self.cfg, self.texts[idx], essay_text=self.essay_text[idx])
        # essay_ids = prepare_essay_input(self.cfg, self.essay_text[idx])
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        # label = self.labels[idx]
        
        # input_ids = [self.tokenizer.cls_token_id] + ids
        # if len(input_ids) > self.cfg.max_len - 1:
        #     input_ids = input_ids[: self.cfg.max_len - 1]
        # mask = [1] * len(input_ids) #TODO modify prepare_input to return input_ids and mask

        # return {
        #     "ids": torch.tensor(input_ids, dtype=torch.long),
        #     "mask": torch.tensor(mask, dtype=torch.long),
        #     # "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        #     "targets": label,
        # }

        # TODO
        # input_ids = input_ids[:self.cfg.max_len]
        # essay_ids = essay_ids[:self.cfg.max_len]

        # return input_ids, essay_ids, label
        return input_ids, label


class FeedbackDataset123:
    def __init__(self, samples, cfg, tokenizer):
        self.samples = samples
        self.cfg = cfg
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        ids = self.samples[idx]["input_ids"]
        label = self.samples[idx]["label"]

        input_ids = [self.tokenizer.cls_token_id] + ids

        if len(input_ids) > self.cfg.max_len - 1:
            input_ids = input_ids[: self.cfg.max_len - 1]

        input_ids = input_ids + [self.tokenizer.sep_token_id]
        mask = [1] * len(input_ids)

        return {
            "ids": input_ids,
            "mask": mask,
            # "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": label,
        }

# Model

In [ ]:
from sklearn import metrics

In [ ]:
!pip3 install bitsandbytes-cuda111
import bitsandbytes as bnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class Collate:
    def __init__(self, tokenizer, args):
        self.tokenizer = tokenizer
        self.args = args

    def __call__(self, batch):
        output = dict()
        output["ids"] = [sample["ids"] for sample in batch]
        output["mask"] = [sample["mask"] for sample in batch]
        output["targets"] = [sample["targets"] for sample in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["ids"]])

        # add padding
        if self.tokenizer.padding_side == "right":
            output["ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["ids"]]
            output["mask"] = [s + (batch_max - len(s)) * [0] for s in output["mask"]]
        else:
            output["ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["ids"]]
            output["mask"] = [(batch_max - len(s)) * [0] + s for s in output["mask"]]

        # convert to tensors
        output["ids"] = torch.tensor(output["ids"], dtype=torch.long)
        output["mask"] = torch.tensor(output["mask"], dtype=torch.long)
        output["targets"] = torch.tensor(output["targets"], dtype=torch.long)

        return output


class AttentionHead(nn.Module):
    def __init__(self, in_size: int = 768, hidden_size: int = 512) -> None:
        super().__init__()
        self.W = nn.Linear(in_size, hidden_size)
        self.V = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(0.1)

    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)
        output = self.dropout(context_vector)
        return output


class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings




class MyFeedbackModel(nn.Module):
    def __init__(self, cfg, num_train_steps, num_labels=3):
        super().__init__()
        self.cfg = cfg
        self.learning_rate = cfg.encoder_lr
        self.model_name = cfg.model
        self.num_train_steps = num_train_steps
        self.num_labels = num_labels

        hidden_dropout_prob = CFG.fc_dropout
        layer_norm_eps = cfg.eps

        # config = AutoConfig.from_pretrained(self.model_name)
        config = AutoConfig.from_pretrained(cfg.model)
        # config.update(
        #     {
        #         "output_hidden_states": True,
        #         "hidden_dropout_prob": hidden_dropout_prob,
        #         "layer_norm_eps": layer_norm_eps,
        #         "add_pooling_layer": False,
        #         "num_labels": self.num_labels,
        #     }
        # )

        self.config = config
        self.transformer = AutoModel.from_pretrained(self.model_name, config=self.config)
        # self.transformer = AutoModel.from_config(self.config)
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.15)
        self.dropout3 = nn.Dropout(0.2)
        self.dropout4 = nn.Dropout(0.25)
        self.dropout5 = nn.Dropout(0.3)
        self.discourse_ffl = nn.Linear(config.hidden_size, config.hidden_size)
        # self.attention = AttentionHead(in_size=config.hidden_size, hidden_size=config.hidden_size)
        # self.pooler = MeanPooling()
        self.output = nn.Linear(config.hidden_size, self.num_labels)
    
    def optimizer_scheduler(self):
        param_optimizer = list(self.named_parameters())
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {
                "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                "weight_decay": 0.001,
            },
            {
                "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        opt = AdamW(optimizer_parameters, lr=self.learning_rate)
        def get_scheduler(cfg, optimizer, num_train_steps):
            if cfg.scheduler == 'linear':
                scheduler = get_linear_schedule_with_warmup(
                    optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
                )
            elif cfg.scheduler == 'cosine':
                scheduler = get_cosine_schedule_with_warmup(
                    optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
                )
            return scheduler
        sch = get_scheduler(self.cfg, opt, self.num_train_steps)
        return opt, sch

    def loss(self, outputs, targets):
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(outputs, targets)
        return loss

    def monitor_metrics(self, outputs, targets):
        device = targets.get_device()
        mll = metrics.log_loss(
            targets.cpu().detach().numpy(),
            outputs.cpu().detach().numpy(),
            labels=[0, 1, 2],
        )
        return {"mll": torch.tensor(mll, device=device)}

    # def forward(self, ids, mask=None, essay_ids=None, essay_mask=None, token_type_ids=None, targets=None):
    def forward(self, ids, mask=None, token_type_ids=None, targets=None):
        if not (token_type_ids is None):
            transformer_out = self.transformer(ids, mask, token_type_ids)
            # essay_out = self.essay_model(essay_ids, essay_mask)
        elif not (mask is None):
            transformer_out = self.transformer(ids, mask)
            # essay_out = self.essay_model(essay_ids, essay_mask)
        else:
            transformer_out = self.transformer(ids)
            # essay_out = self.transformer(essay_ids)
        # sequence_output = transformer_out.pooler_output
        transformer_output = transformer_out.last_hidden_state[:, 0, :]
        transformer_output = torch.squeeze(transformer_output)
        # transformer_output = self.attention(transformer_output)
        sequence_output = self.discourse_ffl(self.dropout(transformer_output))

        # sequence_output = torch.cat((sequence_output, transformer_output), dim=2)
        # sequence_output = torch.add(sequence_output, transformer_output)
        # sequence_output = self.pooler(sequence_output, mask)
        # sequence_output = self.dropout(sequence_output)

        # logits = self.output(sequence_output)
        logits1 = self.output(self.dropout1(sequence_output))
        logits2 = self.output(self.dropout2(sequence_output))
        logits3 = self.output(self.dropout3(sequence_output))
        logits4 = self.output(self.dropout4(sequence_output))
        logits5 = self.output(self.dropout5(sequence_output))

        logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
        # logits = torch.softmax(logits, dim=-1)
        # loss = 0

        if targets is not None:
            # loss = self.loss(logits, targets)
            loss1 = self.loss(logits1, targets)
            loss2 = self.loss(logits2, targets)
            loss3 = self.loss(logits3, targets)
            loss = (loss1 + loss2 + loss3) / 3
            # metric = self.monitor_metrics(logits, targets)
            return logits, loss

        return logits



In [ ]:
from torch.autograd import Variable
from torch.nn.modules.loss import _WeightedLoss


class SmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    def k_one_hot(self, targets:torch.Tensor, n_classes:int, smoothing=0.0):
        with torch.no_grad():
            targets = torch.empty(size=(targets.size(0), n_classes),
                                  device=targets.device) \
                                  .fill_(smoothing /(n_classes-1)) \
                                  .scatter_(1, targets.data.unsqueeze(1), 1.-smoothing)
        return targets

    def reduce_loss(self, loss):
        return loss.mean() if self.reduction == 'mean' else loss.sum() if self.reduction == 'sum' else loss

    def forward(self, inputs, targets):
        assert 0 <= self.smoothing < 1

        targets = self.k_one_hot(targets, inputs.size(-1), self.smoothing)
        log_preds = F.log_softmax(inputs, -1)

        if self.weight is not None:
            log_preds = log_preds * self.weight.unsqueeze(0)

        return self.reduce_loss(-(targets * log_preds).sum(dim=-1))


class LabelSmoothing(nn.Module):
    """NLL loss with label smoothing.
    """
    def __init__(self, smoothing=0.0):
        """Constructor for the LabelSmoothing module.
        :param smoothing: label smoothing factor
        """
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing

    def forward(self, x, target):
        logprobs = torch.nn.functional.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

In [ ]:
class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1., emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                if name in self.backup:
                    param.data = self.backup[name]
            self.backup = {}

# Helper Functions

In [ ]:
def add_inputs(df):
    df['inputs'] = df.discourse_type + ' ' + tokenizer.sep_token + ' ' + df.discourse_text
    return df

metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# ====================================================
# Helper functions
# ====================================================
def get_score(y_true, y_pred):
    log_loss = metrics.log_loss
    # trues = []
    # a1, a2, a3 = [1., 0., 0.], [0., 1., 0.], [0., 0., 1.]
    # for true in y_true:
    #     t = a1 if true == 0 else a2 if true == 1 else a3
    #     trues.append(t)
    score = log_loss(y_true, y_pred)
    return score

def get_score_1(y_true, y_pred):
    log_loss = metrics.log_loss
    y_true1, y_true2, y_true3 = [], [], []
    y_pred1, y_pred2, y_pred3 = [], [], []

    for true, pred in zip(y_true, y_pred):
        if true == 0:
            y_true1.append(1.)
            y_true2.append(0.)
            y_true3.append(0.)
        elif true == 1:
            y_true1.append(0.)
            y_true2.append(1.)
            y_true3.append(0.)
        else:
            y_true1.append(0.)
            y_true2.append(0.)
            y_true3.append(1.)
        y_pred1.append(pred[0])
        y_pred2.append(pred[1])
        y_pred3.append(pred[2])
    # print('pred', y_pred)
    # print('y_true1', y_true1)
    # print('y_pred1', y_pred1)
    score1 = log_loss(y_true1, y_pred1)
    score2 = log_loss(y_true2, y_pred2)
    score3 = log_loss(y_true3, y_pred3)
    return (score1 + score2 + score3) / 3


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))



def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    fgm = FGM(model)

    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        batch_size = labels.size(0)
        labels = labels.to(device)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs['input_ids'], inputs['attention_mask'])
        loss = criterion(y_preds, labels)

        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        
        # -------------------
        # FGM
        fgm.attack()
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs['input_ids'], inputs['attention_mask'])
        loss_adv = criterion(y_preds, labels)
        loss_adv.backward()
        fgm.restore()
        # -------------------
        
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0])) 
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
        torch.cuda.empty_cache()
        gc.collect()
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        batch_size = labels.size(0)
        labels = labels.to(device)
        with torch.no_grad():
            y_preds = model(inputs['input_ids'], inputs['attention_mask'])
        loss = criterion(y_preds, labels)

        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        # preds.append(y_preds.sigmoid().to('cpu').numpy())
        pred_np = y_preds.sigmoid().to('cpu').numpy()
        if len(pred_np.shape) == 1:
            pred_np = np.array([pred_np.tolist()])
        preds.append(pred_np)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    # predictions = np.concatenate(predictions)
    return losses.avg, predictions


def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [ ]:
class Preds:
    pred=[]
    label=[]

In [ ]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold, pin_memory=False):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds['discourse_effectiveness'].values
    train_dataset = FeedbacksDataset(CFG, train_folds)
    valid_dataset = FeedbacksDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=pin_memory, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=pin_memory, drop_last=False)
    
    # number of steps for training
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = MyFeedbackModel(CFG, num_train_steps)

    torch.save(model.config, OUTPUT_DIR+'config.pth')
    torch.cuda.empty_cache()
    gc.collect()
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.transformer.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.transformer.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "transformer" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    # criterion = nn.BCEWithLogitsLoss(reduction="mean")
    # criterion = nn.CrossEntropyLoss(reduction="mean")
    criterion = SmoothCrossEntropyLoss(smoothing=0.1)
    
    best_score = 10.

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        
        # scoring
        Preds.label, Preds.pred = valid_labels, predictions
        score = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_loss": avg_loss, 
                       f"[fold{fold}] avg_val_loss": avg_val_loss,
                       f"[fold{fold}] score": score})
        
        if best_score > score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")
        
        torch.cuda.empty_cache()
        gc.collect()

    predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    # valid_folds['Ineffective'] = predictions[0]
    # valid_folds['Adequate'] = predictions[1]
    # valid_folds['Effective'] = predictions[2]

    gc.collect()
    
    return valid_folds, best_score

In [ ]:
if __name__ == '__main__':
    if CFG.train:
        oof_df = pd.DataFrame()
        scores = []
        for fold in range(CFG.n_fold):
            torch.cuda.empty_cache()
            if fold in CFG.trn_fold:
                torch.cuda.empty_cache()
                _oof_df, score = train_loop(train_df, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                LOGGER.info(f'Score: {score:<.4f}')
                scores.append(score)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        score = np.mean(scores)
        LOGGER.info(f'Score: {score:<.4f}')
        oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        
    if CFG.wandb:
        wandb.finish()

========== fold: 0 training ==========
Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

Epoch: [1][0/9060] Elapsed 0m 1s (remain 246m 25s) Loss: 1.1198(1.1198) Grad: 341931.2812  LR: 0.00000040  
Epoch: [1][100/9060] Elapsed 2m 1s (remain 180m 6s) Loss: 0.6283(0.9891) Grad: 67108.5000  LR: 0.00002000  
Epoch: [1][200/9060] Elapsed 4m 1s (remain 177m 38s) Loss: 0.9354(0.9631) Grad: 258247.5781  LR: 0.00002000  
Epoch: [1][300/9060] Elapsed 6m 1s (remain 175m 28s) Loss: 1.3436(0.9640) Grad: 136550.2344  LR: 0.00002000  
Epoch: [1][400/9060] Elapsed 8m 1s (remain 173m 24s) Loss: 1.4339(0.9450) Grad: 131854.8438  LR: 0.00002000  
Epoch: [1][500/9060] Elapsed 10m 1s (remain 171m 19s) Loss: 0.6855(0.9339) Grad: 29234.9902  LR: 0.00002000  
Epoch: [1][600/9060] Elapsed 12m 1s (remain 169m 21s) Loss: 1.1170(0.9238) Grad: 30170.7520  LR: 0.00001999  
Epoch: [1][700/9060] Elapsed 14m 1s (remain 167m 16s) Loss: 0.6593(0.9177) Grad: 33706.2539  LR: 0.00001999  
Epoch: [1][800/9060] Elapsed 16m 1s (remain 165m 12s) Loss: 0.9424(0.9179) Grad: 33872.5352  LR: 0.00001999  
Epoch: [1][900

Epoch 1 - avg_train_loss: 0.8219  avg_val_loss: 0.9104  time: 11305s
Epoch 1 - Score: 0.7912
Epoch 1 - Save Best Score: 0.7912 Model


EVAL: [2265/2266] Elapsed 8m 1s (remain 0m 0s) Loss: 0.4277(0.9104) 
Epoch: [2][0/9060] Elapsed 0m 1s (remain 171m 49s) Loss: 1.1249(1.1249) Grad: 871511.2500  LR: 0.00001811  
Epoch: [2][100/9060] Elapsed 2m 1s (remain 180m 20s) Loss: 0.4725(0.8225) Grad: 100689.0312  LR: 0.00001807  
Epoch: [2][200/9060] Elapsed 4m 2s (remain 177m 58s) Loss: 0.4347(0.8186) Grad: 54030.5938  LR: 0.00001802  
Epoch: [2][300/9060] Elapsed 6m 2s (remain 175m 43s) Loss: 1.1740(0.7988) Grad: 270054.4688  LR: 0.00001798  
Epoch: [2][400/9060] Elapsed 8m 2s (remain 173m 33s) Loss: 0.8872(0.7740) Grad: 105641.7344  LR: 0.00001794  
Epoch: [2][500/9060] Elapsed 10m 2s (remain 171m 26s) Loss: 0.4268(0.7676) Grad: 27927.9160  LR: 0.00001790  
Epoch: [2][600/9060] Elapsed 12m 1s (remain 169m 18s) Loss: 1.0530(0.7704) Grad: 125183.3516  LR: 0.00001786  
Epoch: [2][700/9060] Elapsed 14m 1s (remain 167m 13s) Loss: 0.7971(0.7693) Grad: 119629.8594  LR: 0.00001781  
Epoch: [2][800/9060] Elapsed 16m 1s (remain 165m 8s)

Epoch 2 - avg_train_loss: 0.7698  avg_val_loss: 0.8446  time: 11504s
Epoch 2 - Score: 0.7660
Epoch 2 - Save Best Score: 0.7660 Model


EVAL: [2265/2266] Elapsed 8m 3s (remain 0m 0s) Loss: 0.4041(0.8446) 
Epoch: [3][0/9060] Elapsed 0m 1s (remain 172m 35s) Loss: 1.1986(1.1986) Grad: 833074.5625  LR: 0.00001311  
Epoch: [3][100/9060] Elapsed 2m 3s (remain 183m 10s) Loss: 1.1877(0.7142) Grad: 431668.1250  LR: 0.00001304  
Epoch: [3][200/9060] Elapsed 4m 7s (remain 181m 33s) Loss: 0.6996(0.7106) Grad: 232864.4219  LR: 0.00001298  
Epoch: [3][300/9060] Elapsed 6m 11s (remain 180m 1s) Loss: 1.1487(0.7101) Grad: 419086.4062  LR: 0.00001291  
Epoch: [3][400/9060] Elapsed 8m 15s (remain 178m 12s) Loss: 0.4351(0.7002) Grad: 48053.5234  LR: 0.00001285  


In [ ]:
#  a = train_df[train_df['fold'] == fold].reset_index(drop=True)
# v = a['discourse_effectiveness'].values

In [ ]:
# l = [[0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0]]
# a = [[0.05382654, 0.8674609,  0.07871257], [0.05382654, 0.8674609,  0.07871257], [0.05382654, 0.8674609,  0.07871257], [0.05382654, 0.8674609,  0.07871257], [0.05382654, 0.8674609,  0.07871257]]

# metrics.log_loss(l, a)

In [ ]:
# len(Preds.pred[2264].shape)
# np.array([Preds.pred[2264].tolist()])

In [ ]:
# # torch.mean(torch.tensor(Preds.pred), dim=1).numpy()
# # predsss = np.concatenate(Preds.pred)
# for true, pred in zip(Preds.label, Preds.pred):
#     p = pred.tolist()
#     print('pred -> ', pred)
#     print('p ->', p)
#     break

In [ ]:
# get_score(Preds.label, torch.mean(torch.tensor(Preds.pred), dim=1).numpy())